# 大众点评评价情感分析~
先上结果：

| 糖水店的评论文本                             | 模型预测的情感评分 |
| :------------------------------------------- | :----------------- |
| '糖水味道不错，滑而不腻，赞一个，下次还会来' | 0.91               |
| '味道一般，没啥特点'                         | 0.52               |
| '排队老半天，环境很差，味道一般般'           | 0.05               |

模型的效果还可以的样子，yeah~接下来我们好好讲讲怎么做的哈，我们通过爬虫爬取了大众点评广州8家最热门糖水店的3W条评论信息以及评分作为训练数据，前面的分析我们得知*样本很不均衡*。接下来我们的整体思路就是：文本特征处理(分词、去停用词、TF-IDF)—机器学习建模—模型评价。

我们先不处理样本不均衡问题，直接建模后查看结果，接下来我们再按照两种方法处理样本不均衡，对比结果。

### 数据读入和探索

In [67]:
import pandas as pd
from matplotlib import pyplot as plt
import jieba
import joblib
data = pd.read_csv('data.csv')
data.head()

,cus_id,comment_time,comment_star,cus_comment,kouwei,huanjing,fuwu,shopID,stars,year,month,weekday,hour,comment_len
0,wt92,2024-05-27 23:35:00,sml-str45,[ 薄荷 ] 环境 ： \n 店面 虽小 ， 五脏俱全 ， 还是 老 店铺 的 那种 味道 风格 ， 格外 亲切 。 \n 「 凤凰 炒奶 」 新鲜 滚烫 的 鲜牛奶 加入 一个 鸡蛋黄 ， 通过 自己 不停 的 搅拌 均匀 变成 金黄色 的 成品 。 入口 润滑 浓香 扑鼻 。 「 椰汁 西米 冰 」 醇香 椰汁 搭配 饱满 西米露 ， 夏日 解暑 佳品 。 \n \n \n \t \t 收起 评价,4,4,4,521698,4.5,2024,5,0,23,179
1,悟道,2024-05-26 18:04:00,sml-str45,红豆 双皮奶 简直 太绝 了 ！ 奶皮 厚实 ， 入口 即 化 ， 奶香 浓郁 醇厚 。 搭配 上 香甜 的 红豆 ， 口感 丰富 ， 甜而不腻 ， 每 一口 都 让 人 陶醉 。 不愧 是 招牌 甜品 ， 真的 强烈推荐 ， 吃 过 就 会 被 它 深深 折服 。 \n 甜品 的 种类 丰富 多样 ， 每一款 都 独具 风味 。 无论是 双皮奶 的 细腻 嫩 滑 、 姜撞奶 的 独特 口感 ， 还是 其他 甜品 的 香甜 滋味 ， 都 让 人 欲罢不能 ， 每次 有 朋友 来 这边 都 带来 试下 \n 绝对 值得 一试 ！ \n \n \n \t \t 收起 评价,5,2,2,521698,4.5,2024,5,6,18,229
2,重度碳水爱好者,2024-05-26 20:32:00,sml-str35,[ 薄荷 ] 环境 ： \n 在 小道 里面 ， 门面 不 大 ， 但是 有 很多 座位 ， 稍微 排 排队 ， 翻桌 还是 挺快 的 \n [ 樱花 ] 性价比 ： \n 说实话 份量 挺大 的 ， 价格 不 贵 ， 就是 吃不完 哈哈哈 哈哈哈 \n 「 红豆 双皮奶 」 这个 双皮奶 我 很 爱 ， 比英 姐家 好吃 ！ \n 「 杏仁 炒奶 」 这个 炒奶 只能 做 热 的 ， 杏仁 味十足 ， 特别 特别 的 丝 滑 ， 很 好吃 ， 也 没有 特别 甜 ， 但 我 就是 吃 了 四五口 就 容易 腻 了 但是 是 好吃 的 \n 「 沙湾 姜撞奶 」 姜 味道 还是 很足 的 ， 很丝滑 \n 「 菠萝 奶冰 」 我 朋友 说 很 好喝 ， 很 像 那个 旺仔 牛奶 味道 。 \n 总体 就是 路过 尝尝 ， 别点 多 ！ \n \n \n \t \t 收起 评价,3,3,3,521698,3.5,2024,5,6,20,286
3,安然Evol,2024-04-27 12:34:00,sml-str50,[ 薄荷 ] 环境 ： \n 排 小队 ， 都 是 本地人 ， 开 在 老式 居民区 楼下 ， 是 当地人 爱来 的 点 。 盲点 一个 都 没 出错 ， 非常 好吃 。 \n \n 口味 ： \n 「 锅蛋 芝麻糊 」 芝麻 和 奶 的 香味 混合 着 ， 真的 要不是 吃 的 太饱 ， 可以 再 来 一碗 。 \n 「 芒果 双皮奶 」 他家 的 点 的 更 多 的 是 红豆 双皮奶 ， 我 爱人 偏爱 芒果 ， 所以 点 了 芒果 的 ， 双皮奶 也 算是 广东 家喻户晓 的 甜品 ， 单奶 口感 十分 浓郁 ， 搭配 芒果 很 有 层次 。 \n 「 木瓜 炖雪耳 」 清凉 美容 ， 超级 好喝 \n \n \n \t \t 收起 评价,5,4,3,521698,5.0,2024,4,5,12,247
4,歪蜜要来,2024-04-10 15:24:00,sml-str40,[ 薄荷 ] 环境 : \n 在 一个 小区 里面 的 ， 看来 是 酒香 不怕 巷子深 哈 。 \n \n [ 樱花 ] 性价比 : \n 不贵 的 ， 碗 挺 大 ， 实在 。 \n 「 凤凰 炒奶 」 据说 是 阿姨 们 现场 炒制 的 ， 我 坐在 外面 的 座位 也 没 怎么 看到 厨房 里面 。 \n 「 驰名 双皮奶 」 双皮奶 碗 挺 大 的 ， 凉凉的 很 解腻 。 \n 整体 推荐 。 \n \n \n \t \t 收起 评价,4,4,4,521698,4.0,2024,4,2,15,183


### 构建标签值

大众点评的评分分为1-5分，1-2为差评，4-5为好评，3为中评，因此我们把1-2记为0,4-5记为1,3为中评，对我们的情感分析作用不大，丢弃掉这部分数据，但是可以作为训练语料模型的语料。我们的情感评分可以转化为标签值为1的概率值，这样我们就把情感分析问题转为文本分类问题了。

In [68]:
#构建label值
def zhuanhuan(score):
    if score > 3:
        return 1
    elif score < 3:
        return 0
    else:
        return None
    
#特征值转换
data['target'] = data['stars'].map(lambda x:zhuanhuan(x))
data_model = data.dropna()

### 文本特征处理

中文文本特征处理，需要进行中文分词，jieba分词库简单好用。接下来需要过滤停用词，网上能够搜到现成的。最后就要进行文本转向量，有词库表示法、TF-IDF、word2vec等，这篇文章作了详细介绍，推荐一波 https://zhuanlan.zhihu.com/p/44917421

这里我们使用sklearn库的TF-IDF工具进行文本特征提取。

In [69]:
#切分测试集、训练集
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_model['cus_comment'], data_model['target'], random_state=3, test_size=0.25)

#引入停用词
infile = open("stopwords.txt",encoding='utf-8')
stopwords_lst = infile.readlines()
stopwords = [x.strip() for x in stopwords_lst]

#中文分词
def fenci(train_data):
    words_df = train_data.apply(lambda x:' '.join(jieba.cut(x)))
    return words_df
 
x_train[:5]

51                                                                                                                                                                                                                                                芒果 双皮奶 好吃 ， 正是 芒果 季节 ， 双皮奶 嫩 滑 清香 ， 香芋 炒奶 真的 爱 了 ， 奶糊 里面 有 香芋 粒粒 ， 香芋 雪球 也 好吃 。
55      大块 的 荔浦 芋头 ， 一口 一个 太 满足 了 「 香芋 糖水 」 ， 虽然 不是 热门 产品 ， 但 这个 芋头 绵软 香甜 ， 推荐 。 \n \n 「 黑芝麻糊 」 很 细腻 ， 芝麻 香味 浓 ， 因为 是 夏天 去 的 有点 热 ， 加上 芝麻糊 是 热 的 ， 所以 感觉 有点 甜腻 。 \n \n 「 凤凰 炒奶 」 阿姨 现场 炒 的 ， 蛋 香味 、 奶 香味 混合 的 汤汁 怎么 能 不 好喝 呢 ！ \n \n 不过 阿姨 有 一点点 凶 呢 \n \n \n \t \t                                                         收起 评价
59                                                        能 吃 到 这样 传统 的 糖水 的 店 ， 现在 不太多 了 ， 而且 这种 老店 更是 少之又少 ， 可能 老街坊 才 懂得 欣赏 吧 ， 一定 要 坚持下去 啊 这里 的 特色 是 奶类 ， 这次 吃 的 是 凤凰 奶糊 ， 还是 以前 的 味道 ， 总体 感觉 会 比 某花 好 ， 不会 太甜 。 另外 要 推介 它 的 香芋 蛋花 奶 ， 也 是 绝 绝子 喔 \n \n \n \t \t                                                         收起 评价
47    老字号 糖水 铺 ， 但 如果 不是 朋友 带路 ， 我

In [70]:
#使用TF-IDF进行文本转向量处理
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(stop_words=stopwords, max_features=3000, ngram_range=(1,2))
tv.fit(x_train)

C:\Users\17662\anaconda3\envs\gettext\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'lex', 'll', 'mon', 'null', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(


TfidfVectorizer(max_features=3000, ngram_range=(1, 2),
                stop_words=['!', '"', '#', '$', '%', '&', "'", '(', ')', '*',
                            '+', ',', '-', '--', '.', '..', '...', '......',
                            '...................', './', '.一', '记者', '数', '年',
                            '月', '日', '时', '分', '秒', '/', ...])

### 机器学习建模

特征和标签已经准备好了，接下来就是建模了。这里我们使用文本分类的经典算法朴素贝叶斯算法，而且朴素贝叶斯算法的计算量较少。特征值是评论文本经过TF-IDF处理的向量，标签值评论的分类共两类，好评是1，差评是0。情感评分为分类器预测分类1的概率值。

In [71]:
#计算分类效果的准确率
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score, f1_score
classifier = MultinomialNB()
classifier.fit(tv.transform(x_train), y_train)
classifier.score(tv.transform(x_test), y_test)

0.9333333333333333

In [72]:
#计算分类器的AUC值
y_pred = classifier.predict_proba(tv.transform(x_test))[:,1]
roc_auc_score(y_test,y_pred)

1.0

In [73]:
#计算一条评论文本的情感评分
def ceshi(model,strings):
    strings_fenci = fenci(pd.Series([strings]))
    return float(model.predict_proba(tv.transform(strings_fenci))[:,1])

In [87]:
#从大众点评网找两条评论来测试一下
test1 = '很好吃，环境好，所有员工的态度都很好，上菜快，服务也很好，味道好吃，都是用蒸馏水煮的，推荐，超好吃' #5星好评
test2 = '糯米外皮不绵滑，豆沙馅粗躁，没有香甜味。12元一碗不值。' #1星差评
print('好评实例的模型预测情感得分为{}\n差评实例的模型预测情感得分为{}'.format(ceshi(classifier,test1),ceshi(classifier,test2)))

好评实例的模型预测情感得分为0.9461026921771922
差评实例的模型预测情感得分为0.9255170920279938


可以看出，准确率和AUC值都非常不错的样子，但点评网上的实际测试中，5星好评模型预测出来了，1星差评缺预测错误。为什么呢？我们查看一下**混淆矩阵**

In [88]:
from sklearn.metrics import confusion_matrix
y_predict = classifier.predict(tv.transform(x_test))
cm = confusion_matrix(y_test, y_predict)
cm

array([[ 0,  1],
       [ 0, 14]], dtype=int64)

可以看出，**负类的预测非常不准**，433单准确预测为负类的只有15.7%，应该是由于**数据不平衡**导致的，模型的默认阈值为输出值的中位数。比如逻辑回归的输出范围为[0,1]，当某个样本的输出大于0.5就会被划分为正例，反之为反例。在数据的类别不平衡时，采用默认的分类阈值可能会导致输出全部为正例，产生虚假的高准确度，导致分类失败。

处理样本不均衡问题的方法，首先可以选择调整阈值，使得模型对于较少的类别更为敏感，或者选择合适的评估标准，比如ROC或者F1，而不是准确度（accuracy）。另外一种方法就是通过采样（sampling）来调整数据的不平衡。其中欠采样抛弃了大部分正例数据，从而弱化了其影响，可能会造成偏差很大的模型，同时，数据总是宝贵的，抛弃数据是很奢侈的。另外一种是过采样，下面我们就使用过采样方法来调整。

### 过采样（单纯复制）

单纯的重复了反例，因此会过分强调已有的反例。如果其中部分点标记错误或者是噪音，那么错误也容易被成倍的放大。因此最大的风险就是对反例过拟合。

In [89]:
data['target'].value_counts()

target
1.0    53
0.0     5
Name: count, dtype: int64

In [90]:
#把0类样本复制10次，构造训练集
index_tmp = y_train==0
y_tmp = y_train[index_tmp]
x_tmp = x_train[index_tmp]
x_train2 = pd.concat([x_train,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp])
y_train2 = pd.concat([y_train,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp])

In [95]:
#使用过采样样本(简单复制)进行模型训练，并查看准确率
clf2 = MultinomialNB()
clf2.fit(tv.transform(x_train2), y_train2)
y_pred2 = clf2.predict_proba(tv.transform(x_test))[:,1]
roc_auc_score(y_test,y_pred2)
joblib.dump(clf2, 'naive_bayes_model.pkl')
joblib.dump(tv, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [92]:
#查看此时的混淆矩阵
y_predict2 = clf2.predict(tv.transform(x_test))
cm = confusion_matrix(y_test, y_predict2)
cm

array([[ 1,  0],
       [ 3, 11]], dtype=int64)

可以看出，即使是简单粗暴的复制样本来处理样本不平衡问题，负样本的识别率大幅上升了，变为77%，满满的幸福感呀~我们自己写两句评语来看看

In [93]:
ceshi(clf2,'排队人太多，环境不好，口味一般')

0.3861090070426321

可以看出把0类别的识别出来了，太棒了~

### 过采样（SMOTE算法）

SMOTE（Synthetic minoritye over-sampling technique,SMOTE），是在局部区域通过K-近邻生成了新的反例。相较于简单的过采样，SMOTE降低了过拟合风险，但同时运算开销加大

对SMOTE感兴趣的同学可以看下这篇文章https://www.jianshu.com/p/ecbc924860af

In [83]:
#使用SMOTE进行样本过采样处理
from imblearn.over_sampling import SMOTE
oversampler=SMOTE(random_state=0)
x_train_vec = tv.transform(x_train)
x_resampled, y_resampled = oversampler.fit_resample(x_train_vec, y_train)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 4, n_neighbors = 6

In [84]:
#原始的样本分布
y_train.value_counts()

target
1.0    39
0.0     4
Name: count, dtype: int64

In [85]:
#经过SMOTE算法过采样后的样本分布情况
pd.Series(y_resampled).value_counts()

target
0.0    14920
1.0    14920
Name: count, dtype: int64

我们经过插值，把0类数据也丰富为14923个数据了，这时候正负样本的比例为1:1，接下来我们用平衡后的数据进行训练，效果如何呢，好期待啊~

In [86]:
#使用过采样样本(SMOTE)进行模型训练，并查看准确率
clf3 = MultinomialNB()
clf3.fit(x_resampled, y_resampled)
y_pred3 = clf3.predict_proba(tv.transform(x_test))[:,1]
roc_auc_score(y_test,y_pred3)

ValueError: X has 1607 features, but MultinomialNB is expecting 3000 features as input.

In [ ]:
#查看此时的准确率
y_predict3 = clf3.predict(tv.transform(x_test))
cm = confusion_matrix(y_test, y_predict3)
cm

In [ ]:
#到网上找一条差评来测试一下情感评分的预测效果
test3 = '糯米外皮不绵滑，豆沙馅粗躁，没有香甜味。12元一碗不值。'
ceshi(clf3,test3)

可以看出，使用SMOTE插值与简单的数据复制比起来，AUC率略有提高，实际预测效果也挺好

### 模型评估测试

接下来我们把3W条数据都拿来训练，数据量变多了，模型效果应该会更好

In [ ]:
#词向量训练
tv2 = TfidfVectorizer(stop_words=stopwords, max_features=3000, ngram_range=(1,2))
tv2.fit(data_model['cus_comment'])

#SMOTE插值
X_tmp = tv2.transform(data_model['cus_comment'])
y_tmp = data_model['target']
sm = SMOTE(random_state=0)
X,y = sm.fit_resample(X_tmp, y_tmp)

clf = MultinomialNB()
clf.fit(X, y)
# 保存 TfidfVectorizer 和 朴素贝叶斯模型
joblib.dump(tv2, 'tfidf_vectorizer.pkl')
joblib.dump(clf, 'naive_bayes_model.pkl')
def fenxi(strings):
    strings_fenci = fenci(pd.Series([strings]))
    return float(clf.predict_proba(tv2.transform(strings_fenci))[:,1])

In [82]:
#到网上找一条差评来测试一下
fenxi('糯米外皮不绵滑，豆沙馅粗躁，没有香甜味。12元一碗不值。')

ValueError: X has 1581 features, but MultinomialNB is expecting 3000 features as input.

只用到了简单的机器学习，就做出了不错的情感分析效果，知识的力量真是强大呀，666~
### 后续优化方向

- 使用更复杂的机器学习模型如神经网络、支持向量机等
- 模型的调参
- 行业词库的构建
- 增加数据量
- 优化情感分析的算法
- 增加标签提取等
- 项目部署到服务器上，更好地分享和测试模型的效果